In [1]:
# grep -i "gw01.extranet.frachtwerk.de filterlog" pfsense.log > pfsense.csv
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
pd.set_option('display.max_columns', None)

In [2]:
# Full parquet needs to be build
# df = pd.read_parquet('./data/df_v4.parquet', engine='pyarrow') 
df = pd.read_parquet('./data/df_v4_10000', engine='pyarrow') 

In [3]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True) 

In [5]:
df.dropna(how='all', axis=1, inplace=True) 

In [6]:
def parse_dtype(df, dtypes):
    for col in dtypes.keys():
        df[col] = df[col].astype(dtypes[col])
    return df

In [7]:
#dtypes = {'value':'datetime64[ns]', 'offset':'int', 'length':'int', 'data_length':'int'}
dtypes = {'offset':'float', 'length':'float', 'data_length':'float'}
df = parse_dtype(df, dtypes)

In [8]:
df_port_name = pd.read_csv("/Users/denakarimi/Desktop/WS20:21/BDA/ports-service-mapping-master/port-names.csv")

In [9]:
df_port_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6068 entries, 0 to 6067
Data columns (total 2 columns):
service    6068 non-null object
port       6068 non-null int64
dtypes: int64(1), object(1)
memory usage: 94.9+ KB


In [29]:
df_port_name["port"]=df_port_name.port.apply(str)
df_port_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6068 entries, 0 to 6067
Data columns (total 2 columns):
service    6068 non-null object
port       6068 non-null object
dtypes: object(2)
memory usage: 94.9+ KB


In [35]:
df=df.merge(df_port_name, how="left", left_on="source_port", right_on="port", suffixes=("","_source_name"),copy=False)

In [37]:
df=df.merge(df_port_name, how="left", left_on="destination_port", right_on="port", suffixes=("","_destination_name"),copy=False)

In [38]:
df.head()

,date_time,rule_number,tracker_id,real_interface,reason,action,traffic_direction,ip_version,tos,ecn,ttl,ipv4_id,offset,flags,protocoll_id,protocoll_text,length,source_ip,destination_ip,source_port,destination_port,data_length,tcp_flag,sequence_num,ack,window,options,service,port,service_source_name,port_source_name,service_destination_name,port_destination_name,service_source_name,port_source_name,service_destination_name,port_destination_name,service_destination_name,port_destination_name
0,Sep/7/2020 11:45:50,200,1599469328,vtnet3,match,block,in,4,0x0,None,63,0,0.0,DF,6,tcp,40.0,172.23.0.2,23.54.112.189,51636,443,0.0,R,3551626076,None,0,None,NaN,NaN,NaN,NaN,https,443,NaN,NaN,https,443,https,443
1,Sep/7/2020 11:45:52,200,1599469328,vtnet3,match,block,in,4,0x0,None,63,0,0.0,DF,6,tcp,40.0,172.23.0.2,23.54.112.189,51652,443,0.0,R,900156233,None,0,None,NaN,NaN,NaN,NaN,https,443,NaN,NaN,https,443,https,443
2,Sep/7/2020 11:46:00,200,1599469328,vtnet3,match,block,in,4,0x0,None,63,0,0.0,DF,6,tcp,40.0,172.23.0.2,23.54.112.189,51704,443,0.0,R,319724059,None,0,None,NaN,NaN,NaN,NaN,https,443,NaN,NaN,https,443,https,443
3,Sep/7/2020 11:46:00,200,1599469328,vtnet3,match,block,in,4,0x0,None,63,0,0.0,DF,6,tcp,40.0,172.23.0.2,23.54.112.189,51704,443,0.0,R,319724059,None,0,None,NaN,NaN,NaN,NaN,https,443,NaN,NaN,https,443,https,443
4,Sep/7/2020 11:46:12,200,1599469328,vtnet3,match,block,in,4,0x0,None,63,0,0.0,DF,6,tcp,40.0,172.23.0.2,23.54.112.189,51770,443,0.0,R,3030680876,None,0,None,NaN,NaN,NaN,NaN,https,443,NaN,NaN,https,443,https,443


In [34]:
df.to_csv("/Users/denakarimi/Desktop/WS20:21/BDA/merged_port_names.csv")